In [1]:
import numpy as np
import skimage.io
import skimage.measure
#from scipy.ndimage.measurements import label
import tifffile as tiff
import matplotlib.pyplot as plt

In [2]:
def im_prep(str_pred, str_mask, win_size):
    prediction = skimage.io.imread(str_pred)
    mask = skimage.io.imread(str_mask)
    
    mask = mask[win_size:-win_size, win_size:-win_size]
    mask = np.pad(mask, (win_size, win_size), 'constant')
    prediction = np.pad(prediction, (win_size, win_size), 'constant')
    
    return prediction, mask


In [3]:
wshed_pred, wshed_mask = im_prep('wshed_pred.tif', 'wshed_mask.tif', 15)

In [10]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10,10]

In [ ]:
plt.subplot(1,1,1)

In [ ]:
plt.imshow(wshed_pred)

In [ ]:
plt.imshow(wshed_mask)

In [4]:
print('watershed mask max is: ', wshed_mask.max(), ' shape is: ', wshed_mask.shape)
print('watershed prediction max is: ', wshed_pred.max(), ' shape is: ', wshed_pred.shape)

watershed mask max is:  255  shape is:  (2048, 2048)
watershed prediction max is:  5798  shape is:  (2048, 2048)


In [ ]:
def dice_pixel(pred_input, truth_input):
    pred = np.copy(pred_input)
    truth = np.copy(truth_input)
    
    pred[pred != 0] = 1
    truth[truth != 0] = 1    
    
    true_pos = 0
    false_pos = 0
    false_neg = 0
    
    if pred.shape != truth.shape:
        print('shape of inputs need to match. Shape of prediction is: ', pred.shape, ' Shape of mask is: ', truth.shape)
    
    for y,x in np.ndindex(wshed_pred.shape):
        if((pred[y][x]==1) & (truth[y][x]==1)):
            true_pos += 1
        elif((pred[y][x]==1) & (truth[y][x]==0)):
            false_pos += 1
        elif((pred[y][x]==0) & (truth[y][x]==1)):
            false_neg += 1
    
    dice_pixel = (2*true_pos)/(2*true_pos + false_pos + false_neg)
    
    print('pixelwise dice/F1 score is: ', dice_pixel)
    
    
    
    
    

In [ ]:
dice_pixel(wshed_pred, wshed_mask)

In [5]:
def dice_pixel_quick(pred_input, truth_input):
    
    pred = np.copy(pred_input)
    truth = np.copy(truth_input)
    
    pred[pred != 0] = 1
    truth[truth != 0] = 1    
    
    true_pos = 0
    false_pos = 0
    false_neg = 0
    
    if pred.shape != truth.shape:
        raise ValueError('shape of inputs need to match. Shape of prediction is: ', pred.shape, ' Shape of mask is: ', truth.shape)
    
    if((pred.sum() + truth.sum()) == 0):
        print('arrays are empty. Dice score is technically 1.0')
        return 1.0
    
    intersection = np.logical_and(pred, truth)

    dice_pixel = (2*intersection.sum() / (pred.sum() + truth.sum()))
    jaccard_pixel = dice_pixel/(2-dice_pixel)
    
#    for y,x in np.ndindex(wshed_pred.shape):
#        if((pred[y][x]==1) & (truth[y][x]==1)):
#            true_pos += 1
#        elif((pred[y][x]==1) & (truth[y][x]==0)):
#            false_pos += 1
#        elif((pred[y][x]==0) & (truth[y][x]==1)):
#            false_neg += 1
    
#    dice_pixel = (2*true_pos)/(2*true_pos + false_pos + false_neg)
    
    print('pixelwise dice/F1 index is: ', dice_pixel)
    print('pixelwise jaccard index is: ', jaccard_pixel)
#    return dice_pixel, jaccard_pixel

In [6]:
dice_pixel_quick(wshed_pred, wshed_mask)

pixelwise dice/F1 index is:  0.8812708067220768
pixelwise jaccard index is:  0.7877427459811938


In [53]:
def gen_iou_matrix(pred_input, truth_input, threshold):
    
    pred = np.copy(pred_input)
    truth = np.copy(truth_input)
    
#    print('watershed mask max is: ', truth.max())
#    print('watershed prediction max is: ', pred.max())
    
#    pred[pred != 0] = 1
#    truth[truth != 0] = 1
    
#    pred = skimage.measure.label(pred, connectivity = 2)
    truth = skimage.measure.label(truth, connectivity = 2)
    
    print('watershed mask max post-label is: ', truth.max())
    print('watershed prediction max post-label is: ', pred.max())
    
   # plt.imshow(truth)
    
    iou_matrix = np.zeros((truth.max(), pred.max()))
    
    print('shape of intersection over union matrix is: ', iou_matrix.shape)
    
    for n,m in np.ndindex(iou_matrix.shape):
        
  #      if(n == 0):
  #          print('checkpoint')
        
     #   print(pred.shape, truth.shape)
        
        pred_cpy = np.copy(pred)
        truth_cpy = np.copy(truth)
        
        pred_cpy[pred_cpy != m+1] = 0
        truth_cpy[truth_cpy != n+1] = 0
    
        intersection = np.logical_and(pred_cpy, truth_cpy)
        union = np.logical_or(pred_cpy, truth_cpy)
        
    #    intersection = np.logical_and((pred[pred==(m+1)]), (truth[truth==(n+1)]))
    #    union = np.logical_or((pred[pred==(m+1)]), (truth[truth==(n+1)]))
        
        iou = np.sum(intersection) / np.sum(union)
        
        if iou > threshold:
            iou = 1
        else:
            iou = 0
        
    print('sum is: ', np.sum(iou_matrix))
    
    return iou_matrix, pred.max(), truth.max()

In [54]:
def dice_jaccard_object(pred, truth, threshold=0.5):
    
    iou_matrix, pred_max, truth_max = gen_iou_matrix(pred, truth, 0.5)
    iou_sum = np.sum(iou_matrix)
    dice_object = 2*iou_sum / (2*iou_sum + pred_max-iou_sum + truth_max-iou_sum)
    
    print('object-based dice/F1 score is: ', dice_object)

In [ ]:
dice_jaccard_object(wshed_pred, wshed_mask)

watershed mask max post-label is:  4569
watershed prediction max post-label is:  5798
shape of intersection over union matrix is:  (4569, 5798)
